In [2]:
import os
import warnings

import pandas as pd
from joblib import load
from utils.scaler import *
from sklearn.metrics import *
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

In [3]:
plt.rc('font', family='GULIM')
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

In [4]:
X_test = pd.read_csv('Database/test/X_self_test_norm.csv', index_col=0)
y_test = pd.read_csv('Database/test/y_self_test.csv', index_col=0)
# y_test = inverse_boxcox(y_test, -0.4431146771093272) - 0.000001
# y_test=pd.DataFrame(np.exp(y_test)-1)
target_scaler=load('Database/target_scaler.joblib')
y_test=pd.DataFrame(target_scaler.inverse_transform(y_test), index=y_test.index)

In [29]:
from Module.model import *
backbone = MLP(input_size=57, hidden_size=256, output_size=1).to('cuda')
backbone.load_state_dict(torch.load('Weight/STN001.pth'))

<All keys matched successfully>

In [49]:
backbone.eval()
output=backbone(torch.tensor(X_test.values).to('cuda'))

MLP(
  (MLP): Sequential(
    (0): Linear(in_features=57, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
  )
  (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [50]:
output=pd.DataFrame(output.cpu().detach().numpy())

In [52]:
output[output<0]=0

In [66]:
output.index=y_test.index

# Best-model dict 저장

In [4]:
STN_list = ['STN001', 'STN002', 'STN003', 'STN004', 'STN005', 'STN006', 'STN007', 'STN008', 'STN009', 'STN010',
            'STN011', 'STN012', 'STN013', 'STN014', 'STN015', 'STN016', 'STN017', 'STN018', 'STN019', 'STN020']

STN_list = ['STN001']

dt_files = sorted(filename for filename in os.listdir('File/DT') if filename.endswith('.pkl'))
lgb_files = sorted(filename for filename in os.listdir('File/LGBM') if filename.endswith('.pkl'))
xgb_files = sorted(filename for filename in os.listdir('File/XGB') if filename.endswith('.pkl'))
cat_files = sorted(filename for filename in os.listdir('File/CAT') if filename.endswith('.pkl'))

best_dt_dict = {}
best_lgb_dict = {}
best_xgb_dict = {}
best_cat_dict = {}

for i, STN in enumerate(STN_list):
    best_dt_dict[STN] = load(os.path.join('File/DT', dt_files[i]))
    best_lgb_dict[STN] = load(os.path.join('File/LGBM', lgb_files[i]))
    best_xgb_dict[STN] = load(os.path.join('File/XGB', xgb_files[i]))
    best_cat_dict[STN] = load(os.path.join('File/CAT', cat_files[i]))

# BoxCox 역변환 & 예측값 dict 저장

In [5]:
pred_dt_dict = {}
pred_lgb_dict = {}
pred_xgb_dict = {}
pred_cat_dict = {}

# for i, STN in enumerate(STN_list):
#     pred_dt_dict[STN] = inverse_boxcox(pd.DataFrame(best_dt_dict[STN].predict(X_test)), -0.4431146771093272) - 0.000001
#     pred_lgb_dict[STN] = inverse_boxcox(pd.DataFrame(best_dt_dict[STN].predict(X_test)), -0.4431146771093272) - 0.000001
#     pred_xgb_dict[STN] = inverse_boxcox(pd.DataFrame(best_dt_dict[STN].predict(X_test)), -0.4431146771093272) - 0.000001
#     pred_cat_dict[STN] = inverse_boxcox(pd.DataFrame(best_dt_dict[STN].predict(X_test)), -0.4431146771093272) - 0.000001

# for i, STN in enumerate(STN_list):
#     pred_dt_dict[STN] = pd.DataFrame(np.exp(best_dt_dict[STN].predict(X_test))-1)
#     pred_lgb_dict[STN] = pd.DataFrame(np.exp(best_dt_dict[STN].predict(X_test))-1)
#     pred_xgb_dict[STN] = pd.DataFrame(np.exp(best_dt_dict[STN].predict(X_test))-1)
#     pred_cat_dict[STN] = pd.DataFrame(np.exp(best_dt_dict[STN].predict(X_test))-1)

for i, STN in enumerate(STN_list):
    pred_dt_dict[STN] = pd.DataFrame(best_dt_dict[STN].predict(X_test))
    pred_lgb_dict[STN] = pd.DataFrame(best_dt_dict[STN].predict(X_test))
    pred_xgb_dict[STN] = pd.DataFrame(best_dt_dict[STN].predict(X_test))
    pred_cat_dict[STN] = pd.DataFrame(best_dt_dict[STN].predict(X_test))

# 60개 싱글모델 예측값 DataFrame 저장

In [6]:
prediction_df = pd.DataFrame()
for i, STN in enumerate(STN_list):
    prediction_df = pd.concat(
        [prediction_df,
         pd.DataFrame(pred_dt_dict[STN].values, columns=[f'DT_{STN}']),
         pd.DataFrame(pred_lgb_dict[STN].values, columns=[f'lgb_{STN}']),
         pd.DataFrame(pred_xgb_dict[STN].values, columns=[f'xgb_{STN}']),
         pd.DataFrame(pred_cat_dict[STN].values, columns=[f'cat_{STN}'])], axis=1)
prediction_df.index = X_test.index

# 60개 싱글모델 metric DataFrame 저장

In [7]:
metric_df = pd.DataFrame(columns=['R2', 'RMSE', 'MAE', 'MAPE'])
for i, STN in enumerate(STN_list):
    metric_df = pd.concat(
        [metric_df, pd.DataFrame([r2_score(y_test.values, pred_dt_dict[STN]),
                                  root_mean_squared_error(y_test.values, pred_dt_dict[STN]),
                                  mean_absolute_error(y_test.values, pred_dt_dict[STN]),
                                  mean_absolute_percentage_error(y_test.values, pred_dt_dict[STN])],
                                 index=metric_df.columns, columns=[f'DT_{STN}']).T], axis=0)

    metric_df = pd.concat(
        [metric_df, pd.DataFrame([r2_score(y_test.values, pred_lgb_dict[STN]),
                                  root_mean_squared_error(y_test.values, pred_lgb_dict[STN]),
                                  mean_absolute_error(y_test.values, pred_lgb_dict[STN]),
                                  mean_absolute_percentage_error(y_test.values, pred_lgb_dict[STN])],
                                 index=metric_df.columns, columns=[f'LGB_{STN}']).T], axis=0)

    metric_df = pd.concat(
        [metric_df, pd.DataFrame([r2_score(y_test.values, pred_xgb_dict[STN]),
                                  root_mean_squared_error(y_test.values, pred_xgb_dict[STN]),
                                  mean_absolute_error(y_test.values, pred_xgb_dict[STN]),
                                  mean_absolute_percentage_error(y_test.values, pred_xgb_dict[STN])],
                                 index=metric_df.columns, columns=[f'XGB_{STN}']).T], axis=0)

    metric_df = pd.concat(
        [metric_df, pd.DataFrame([r2_score(y_test.values, pred_cat_dict[STN]),
                                  root_mean_squared_error(y_test.values, pred_cat_dict[STN]),
                                  mean_absolute_error(y_test.values, pred_cat_dict[STN]),
                                  mean_absolute_percentage_error(y_test.values, pred_cat_dict[STN])],
                                 index=metric_df.columns, columns=[f'CAT_{STN}']).T], axis=0)

# CSI 계산

In [68]:
pred_rating_df = output.applymap(lambda x: map_values(x))
truth_rating_df = pd.DataFrame(y_test).applymap(lambda x: map_values(x))
data = pd.concat([pred_rating_df.iloc[:,0], truth_rating_df], axis=1)

In [70]:
mapping = {
    0: "<0.1",
    1: "0.1<=X<0.2",
    2: "0.2<=X<0.5",
    3: "0.5<=X<1",
    4: "1<=X<2",
    5: "2<=X<5",
    6: "5<=X<10",
    7: "10<=X<20",
    8: "20<=X<30",
    9: "X>=30"
}

data['predicted'] = data.iloc[:, 0].map(mapping)
data['observed'] = data.iloc[:, 1].map(mapping)

csi_categories = ["<0.1", "0.1<=X<0.2", "0.2<=X<0.5", "0.5<=X<1", "1<=X<2", "2<=X<5", "5<=X<10", "10<=X<20", "20<=X<30",
                  "X>=30"]
csi_table = pd.DataFrame(0, index=csi_categories, columns=csi_categories)

for index, row in data.iterrows():
    observed = row['observed']
    predicted = row['predicted']
    csi_table.at[observed, predicted] += 1

In [71]:
cal_CSI(csi_table)

0.03618677391327714